In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from selenium import webdriver
import re

In [ ]:
# Initialise Selenium WebDriver
driver = webdriver.Chrome()

all_titles = []
all_authors = []
all_stars = []
all_reviews = []
all_genres = []

# Number of pages you want to scrape
NUM_PAGES = 28

for page_number in range(1, NUM_PAGES + 1):
    url = f"https://www.goodreads.com/list/show/143500.Best_Books_of_the_Decade_2020_s?page={page_number}"
    driver.get(url)
    
    # Wait for page to load
    time.sleep(3)
    
    # Get page source and parse with BeautifulSoup
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find all book rows
    book_rows = soup.select('table.tableList tr')
    
    for row in book_rows:
        # Extract title
        title_elem = row.select_one('a.bookTitle span')
        title = title_elem.get_text(strip=True) if title_elem else 'N/A'

        # Extract author
        author_elem = row.select_one('a.authorName span')
        author = author_elem.get_text(strip=True) if author_elem else 'N/A'

        # Extract and parse rating text
        rating_elem = row.select_one('span.minirating')
        if rating_elem:
            rating_text = rating_elem.get_text(strip=True)
            match = re.search(r'([\d.]+) avg rating — ([\d,]+) ratings', rating_text)
            if match:
                stars = match.group(1)
                reviews = match.group(2)
            else:
                stars = 'N/A'
                reviews = 'N/A'
        else:
            stars = 'N/A'
            reviews = 'N/A'

        # Extract book link
        book_link_elem = row.select_one('a.bookTitle')
        book_link = 'https://www.goodreads.com' + book_link_elem['href'] if book_link_elem else 'N/A'

        # Visit book page to get genres
        if book_link != 'N/A':
            driver.get(book_link)
            time.sleep(3)
            book_html = driver.page_source
            book_soup = BeautifulSoup(book_html, 'html.parser')

            genre_spans = book_soup.select('ul.CollapsableList a.Button--tag span.Button__labelItem')
            genres = [span.get_text(strip=True) for span in genre_spans]
            genres_text = ', '.join(genres)
        else:
            genres_text = 'N/A'

        # Store results
        all_titles.append(title)
        all_authors.append(author)
        all_stars.append(stars)
        all_reviews.append(reviews)
        all_genres.append(genres_text)
        

# Close the browser
driver.quit()

df = pd.DataFrame({
    'Title': all_titles,
    'Author': all_authors,
    'Stars': all_stars,
    'Number of Reviews': all_reviews,
    'Genres': all_genres
})

In [ ]:
len(all_authors)

In [ ]:
df = pd.DataFrame({
    'Title': all_titles,
    'Author': all_authors,
    'Stars': all_stars,
    'Number of Reviews': all_reviews,
    'Genres': all_genres
})

In [ ]:
len(all_titles)

In [ ]:
len(all_stars)

In [ ]:
len(all_reviews)

In [ ]:
len(all_genres)

In [ ]:
len(df)

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df.to_csv('top_books_decade:2020.csv')